In [1]:
import argparse
from adbtools_win import Adbtools
import numpy as np
from PIL import Image
import time
from imgtools import are_images_similar_mse , box2center, img_crop

#from PokemonPosition import PokemonPosition ,pokemon素材
import hashlib
import os
from io import BytesIO

import csv
from collections import defaultdict

import xml.etree.ElementTree as ET
from datetime import datetime
import re

In [2]:
mac1 = "1ab4919e"
mac2 = "2d5757ea" 
mac3 = "18608847" 
mac4 = "58517171"


In [3]:
host = "localhost"  # 替换为你的设备IP
port = 16449  # ADB默认端口
adb_path = r'D:\安卓工具\platform-tools-latest-windows\platform-tools\adb.exe ' # ADB路径
mac = mac1  # 从命令行传入的MAC地址
adb_screenshot = Adbtools(adb_path, host, port, mac )
img=adb_screenshot.adb_screencap_raw()
save_dir="F:/视频截图/"
file_path = os.path.join(save_dir, f"{time.time()}.png")
img.save(file_path)
#Adbtools(adb_path, host, port, mac ).adb_swipe_raw(500,1000,500,100, 300)

设备分辨率: 1080x1920


1080 1920 1


In [4]:
#img
Adbtools(adb_path, host, port, mac ).get_uixml_via_stdout()

设备分辨率: 1080x1920


'<?xml version=\'1.0\' encoding=\'UTF-8\' standalone=\'yes\' ?><hierarchy rotation="0"><node index="0" text="" resource-id="" class="android.widget.FrameLayout" package="tv.danmaku.bili" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" bounds="[0,0][1080,1920]"><node index="0" text="" resource-id="" class="android.widget.LinearLayout" package="tv.danmaku.bili" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" bounds="[0,0][1080,1920]"><node index="0" text="" resource-id="" class="android.widget.FrameLayout" package="tv.danmaku.bili" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false

In [5]:
root = ET.fromstring(Adbtools(adb_path, host, port, mac ).get_uixml_via_stdout())

设备分辨率: 1080x1920


In [6]:
def find_video_covers(xml_str):
    def parse_bounds(bounds_str):
        nums = list(map(int, re.findall(r'\d+', bounds_str)))
        return tuple(nums)  # (x1, y1, x2, y2)

    root = ET.fromstring(xml_str)
    boxes = []

    for node in root.iter('node'):
        if node.attrib.get('class') == 'android.widget.ImageView':
            if node.attrib.get('resource-id') == 'tv.danmaku.bili:id/cover':
                bounds = node.attrib.get('bounds')
                if bounds:
                    box = parse_bounds(bounds)
                    boxes.append(box)
    return boxes


In [7]:
boxes = find_video_covers(Adbtools(adb_path, host, port, mac ).get_uixml_via_stdout())
crops = []  # 用于保存裁剪后的图像对象
for box in boxes:
    crop = img_crop(img, box)
    crops.append(crop)

设备分辨率: 1080x1920


In [8]:
xml_str= Adbtools(adb_path, host, port, mac ).get_uixml_via_stdout()
xml_str

设备分辨率: 1080x1920


'<?xml version=\'1.0\' encoding=\'UTF-8\' standalone=\'yes\' ?><hierarchy rotation="0"><node index="0" text="" resource-id="" class="android.widget.FrameLayout" package="tv.danmaku.bili" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" bounds="[0,0][1080,1920]"><node index="0" text="" resource-id="" class="android.widget.LinearLayout" package="tv.danmaku.bili" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" bounds="[0,0][1080,1920]"><node index="0" text="" resource-id="" class="android.widget.FrameLayout" package="tv.danmaku.bili" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false

In [9]:
from lxml import etree

def extract_gridviews_xpath(xml_str):
    print("xml_str", xml_str)
    root = etree.fromstring(xml_str.encode('utf-8'))
    gridviews = root.xpath(
        "/node[@class='android.widget.FrameLayout']"
        "/node[@class='android.widget.LinearLayout']"
        "/node[@class='android.widget.FrameLayout']"
        "/node[@class='android.widget.FrameLayout']"
        "/node[@class='android.widget.FrameLayout']"
        "/node[@class='android.widget.FrameLayout']"
        "/node[@class='android.widget.FrameLayout']"
        "/node[@class='android.widget.ScrollView'][2]"  # 第二个 ScrollView
        "/node[@class='android.widget.FrameLayout']"
        "/node[@class='android.view.ViewGroup']"
        "/node[@class='androidx.viewpager.widget.ViewPager']"
        "/node[@class='android.view.ViewGroup']"
        "/node[@class='android.widget.FrameLayout']"
        "/node[@class='android.widget.FrameLayout']"
        "/node[@class='android.widget.GridView']"
    )

    result = []
    for gv in gridviews:
        result.append({
            'class': gv.attrib.get('class'),
            'resource-id': gv.attrib.get('resource-id'),
            'bounds': gv.attrib.get('bounds')
        })
    return result

In [10]:
extract_gridviews_xpath(xml_str)

xml_str <?xml version='1.0' encoding='UTF-8' standalone='yes' ?><hierarchy rotation="0"><node index="0" text="" resource-id="" class="android.widget.FrameLayout" package="tv.danmaku.bili" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" bounds="[0,0][1080,1920]"><node index="0" text="" resource-id="" class="android.widget.LinearLayout" package="tv.danmaku.bili" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" bounds="[0,0][1080,1920]"><node index="0" text="" resource-id="" class="android.widget.FrameLayout" package="tv.danmaku.bili" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="fals

[]

In [11]:
root = etree.fromstring(xml_str.encode("utf-8"))

In [12]:
for child in root:
    print(child.attrib.get("class"))


android.widget.FrameLayout


In [37]:
def is_similar_to_4_3(box_str, tol=0.05):
    """
    Check if a bounding box ratio is similar to 4:3.
    
    Args:
        box_str (str): string like '[548,966][1065,1353]'
        tol (float): tolerance for similarity, default ±0.05
    
    Returns:
        ratio (float), is_similar (bool)
    """
    # Extract numbers
    import re
    nums = list(map(int, re.findall(r"\d+", box_str)))
    if len(nums) != 4:
        raise ValueError("Input must contain 4 numbers like [x1,y1][x2,y2]")
    
    x1, y1, x2, y2 = nums
    width = x2 - x1
    height = y2 - y1
    box=(x1, y1, x2, y2)
    if height == 0:
        raise ValueError("Height is zero, invalid box")
    
    ratio = width / height
    target = 4 / 3
    
    return box, abs(ratio - target) <= tol ,ratio

In [97]:
from openai import OpenAI
from io import BytesIO
import os
import base64
from PIL import Image
def image_to_base64(img, format='PNG'):
    buffered = BytesIO()
    img.save(buffered, format=format)
    img_bytes = buffered.getvalue()
    base64_str = base64.b64encode(img_bytes).decode('utf-8')
    return base64_str
def 鉴(crops):
    base64_image = image_to_base64(crops, format='PNG')
    client = OpenAI(
        # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx"
        api_key=os.getenv('DASHSCOPE_API_KEY'),
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )
    completion = client.chat.completions.create(
        model="qwen-vl-max-2025-04-02", # 此处以qwen-vl-max-latest为例，可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/models
        messages=[
            {
                "role": "system",
                "content": [{"type":"text","text": "You are a helpful assistant."}]},
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        # 需要注意，传入Base64，图像格式（即image/{format}）需要与支持的图片列表中的Content Type保持一致。"f"是字符串格式化的方法。
                        # PNG图像：  f"data:image/png;base64,{base64_image}"
                        # JPEG图像： f"data:image/jpeg;base64,{base64_image}"
                        # WEBP图像： f"data:image/webp;base64,{base64_image}"
                        "image_url": {"url": f"data:image/png;base64,{base64_image}"}, 
                    },
                    {
                        "type": "text",
                        "text": "请以标准 JSON 格式输出，用于描述图片中“擦边女主播”的擦边等级。要求如下：\n1. 'level' 字段表示等级，取值 1-10，其中 10 代表 100% 确认的擦边女主播，1 代表完全不可能是擦边视频。\n2. 'tag' 字段包含若干简短标签，用于描述图片的特征。\n3. 重点关注性暗示元素和裸漏部位 \n4. 输出必须严格遵守 JSON 格式。\n示例输出：\n{\n  \"level\": 7,\n  \"tag\": [\"性感姿势\", \"低胸衣服\", \"暗示表情\"]\n}"
                    },                
                ],
            }
        ],
        response_format={"type": "json_object"},
    )
    return completion.choices[0].message.content

In [ ]:
adb_screenshot = Adbtools(adb_path, host, port, mac )
img=adb_screenshot.adb_screencap_raw()
adb_screenshot.adb_swipe_raw(500,500,500,1300,200)
time.sleep(2)
xml_str= adb_screenshot.get_uixml_via_stdout()
root = etree.fromstring(xml_str.encode("utf-8"))
coverdetil_list=[]
for i, child in enumerate(reversed(root.xpath("./node[1]/node[1]/node[1]/node[1]/node[1]/node[1]/node[1]/node[1]/node[3]/node[1]/node[2]/node[1]/node[1]/node[1]/node[1]/node"))):
    #print(i,child.tag, child.attrib)
    for child_children in child.xpath("./node[1]/node[1]"):
        #print(child_children.attrib.get("class"))
        #print(child_children.attrib)
        if child_children.attrib.get("class") != "android.widget.ImageView":
            #print("not ImageView")
            continue
        boxdetail=is_similar_to_4_3(child_children.attrib.get("bounds"))
        if boxdetail[1] == False:
            #print("not similar to 4:3", boxdetail[0])
            continue
        print(boxdetail[0])
        cover_dict={}
        cover_dict['box'] = boxdetail[0]
        cover_dict['vidio_class'] = child.attrib.get("content-desc").split(',')[0]
        cover_dict['vidio_title'] = child.attrib.get("content-desc")
        cover_dict['cover'] = img_crop(img, boxdetail[0])
        coverdetil_list.append(cover_dict)
        #print("blank")
for cover in coverdetil_list:
    try:
        json_str = 鉴(cover['cover'])
    


设备分辨率: 1080x1920


1080 1920 1
(548, 1134, 1065, 1521)


In [ ]:
coverdetil_list[0]['cover'] 

'竖版视频'

In [79]:
import base64
from io import BytesIO

def image_to_base64(img, format='PNG'):
    buffered = BytesIO()
    img.save(buffered, format=format)
    img_bytes = buffered.getvalue()
    base64_str = base64.b64encode(img_bytes).decode('utf-8')
    return base64_str


In [11]:
#os.getenv('DASHSCOPE_API_KEY')

In [94]:
from openai import OpenAI
import os
import base64




# 将xxxx/eagle.png替换为你本地图像的绝对路径
base64_image = image_to_base64(coverdetil_list[0]['cover'] , format='PNG')

client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx"
    api_key=os.getenv('DASHSCOPE_API_KEY'),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)
completion = client.chat.completions.create(
    model="qwen-vl-max-2025-04-02", # 此处以qwen-vl-max-latest为例，可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/models
    messages=[
    	{
    	    "role": "system",
            "content": [{"type":"text","text": "You are a helpful assistant."}]},
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    # 需要注意，传入Base64，图像格式（即image/{format}）需要与支持的图片列表中的Content Type保持一致。"f"是字符串格式化的方法。
                    # PNG图像：  f"data:image/png;base64,{base64_image}"
                    # JPEG图像： f"data:image/jpeg;base64,{base64_image}"
                    # WEBP图像： f"data:image/webp;base64,{base64_image}"
                    "image_url": {"url": f"data:image/png;base64,{base64_image}"}, 
                },
                {
                    "type": "text",
                    "text": "请以标准 JSON 格式输出，用于描述图片中“擦边女主播”的擦边等级。要求如下：\n1. 'level' 字段表示等级，取值 1-10，其中 10 代表 100% 确认的擦边女主播，1 代表完全不可能是擦边视频。\n2. 'tag' 字段包含若干简短标签，用于描述图片的特征。\n3. 重点关注性暗示元素 \n4. 输出必须严格遵守 JSON 格式。\n示例输出：\n{\n  \"level\": 7,\n  \"tag\": [\"性感姿势\", \"低胸衣服\", \"暗示表情\"]\n}"
                },
            ],
        }
    ],
    response_format={"type": "json_object"},
)
print(completion.choices[0].message.content)
#crops[4]

{
  "level": 2,
  "tag": ["动漫角色", "无性暗示元素", "清新风格"]
}
